In [2]:
# !pip install praw
# !pip install pyarrow

In [2]:
import praw
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import zstandard as zstd
from datetime import datetime

### Methodology 1. Reddit praw api

In [7]:
# ### put your personal info 
# reddit = praw.Reddit(client_id='', \
#                      client_secret='', \
#                      user_agent='')

In [8]:
# posts = []
# ml_subreddit = reddit.subreddit('depression')
# for post in ml_subreddit.new(limit=1000):
#     posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
# posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
# posts

In [9]:
# posts = []
# ml_subreddit = reddit.subreddit('depression')
# for post in ml_subreddit.hot(limit=1000):
#     posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
# posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [9]:
# posts

,title,score,id,subreddit,url,num_comments,body,created
0,Our most-broken and least-understood rules is ...,2357,doqwow,depression,https://www.reddit.com/r/depression/comments/d...,177,We understand that most people who reply immed...,1.572361e+09
1,"Regular check-in post, with information about ...",66,16kqyd7,depression,https://www.reddit.com/r/depression/comments/1...,395,Welcome to /r/depression's check-in post - a p...,1.694923e+09
2,This loneliness is getting pretty bad,63,1bm7xyn,depression,https://www.reddit.com/r/depression/comments/1...,18,The drugs don't even help. Why are ppl so mean...,1.711241e+09
3,Why is the younger generation depressed.,16,1bmdi9s,depression,https://www.reddit.com/r/depression/comments/1...,14,1. Cost of living being high.\n2. Toxic work c...,1.711259e+09
4,"I'm 15, suicidal and addicted to weed.",77,1bm2dd1,depression,https://www.reddit.com/r/depression/comments/1...,39,"About 5 months ago, I moved into a group home ...",1.711227e+09
...,...,...,...,...,...,...,...,...
843,I need help,0,1bjjxmj,depression,https://www.reddit.com/r/depression/comments/1...,0,Me and my gf were together for three years but...,1.710958e+09
844,Really struggling,2,1bjegpr,depression,https://www.reddit.com/r/depression/comments/1...,0,I have a complex medical disease(MCAS/POTS). I...,1.710944e+09
845,Depression makes you self-sabotage friendships,2,1bjecph,depression,https://www.reddit.com/r/depression/comments/1...,0,A lot of people seem to avoid others who are d...,1.710944e+09
846,Depressed since childhood,0,1bjjq1n,depression,https://www.reddit.com/r/depression/comments/1...,0,I’ve been trying to find the person I was befo...,1.710957e+09


In [10]:
# posts.title[0]

'Our most-broken and least-understood rules is "helpers may not invite private contact as a first resort", so we\'ve made a new wiki to explain it'

In [11]:
# posts.body[0]

'We understand that most people who reply immediately to an OP with an invitation to talk privately  mean only to help, but this type of response usually leads to either disappointment or disaster.  it usually works out quite differently here than when you say "PM me anytime" in a casual social context.  \n\nWe have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support others here and flag inappropriate content - even more so because we know that so many of you are struggling yourselves.  We\'re hard at work behind the scenes on more information and resources to make it easier to give and get quality help here - this is just a small start.  \n\nOur new wiki page explains in detail why it\'s much better to respond in public comments, at least until you\'ve gotten to know someone.  It will be maintained at /r/depression/wiki/private_contact, and the full text of the current version is below.\n\n*****\n\n###Summary###\n\n**Anyone who, while a

### Methodology 2. Reddit archive data dump (https://the-eye.eu/redarcs/) 

In [10]:
ls

AnxietyDepression_submissions.zst
reddit_data_preprocessing_AnxietyDepression.ipynb
venv/


In [3]:
## step1. load downloaded submission reddit dump 

In [11]:
anxiety_depression_submissions =\
    pd.read_json('AnxietyDepression_submissions.zst',compression=dict(method='zstd', max_window_size=2147483648),\
    lines = True, nrows = 2000000)

In [13]:
# convert the timestamp to utc
anxiety_depression_submissions['created_utc'] =\
    anxiety_depression_submissions['created_utc'].apply(lambda x : datetime.utcfromtimestamp(x))

In [14]:
## step2. download the file into parquet file to speed up data load later stage

In [15]:
anxiety_depression_submissions.to_parquet('anxiety_depression_submissions.parquet', engine='pyarrow')

In [16]:
## step3. read as parquet file 

In [4]:
anxiety_depression_submissions = pd.read_parquet('anxiety_depression_submissions.parquet',engine='pyarrow')
anxiety_depression_submissions

,stickied,from_kind,subreddit_id,created,url,secure_media,from_id,quarantine,score,edited,archived,id,name,link_flair_css_class,gilded,over_18,saved,num_comments,author_flair_css_class,is_self,author,from,permalink,link_flair_text,retrieved_on,ups,media,selftext,hide_score,domain,subreddit,media_embed,author_flair_text,created_utc,secure_media_embed,title,distinguished,thumbnail,downs,locked,contest_mode,post_hint,preview,spoiler,hidden,brand_safe,suggested_sort,thumbnail_height,thumbnail_width,can_gild,view_count,is_video,author_cakeday,approved_at_utc,banned_at_utc,can_mod_post,is_crosspostable,num_crossposts,parent_whitelist_status,whitelist_status,is_reddit_media_domain,pinned,subreddit_type,mod_note,mod_reason_by,mod_reason_title,no_follow,send_replies,subreddit_subscribers,approved_by,author_flair_background_color,author_flair_template_id,author_flair_text_color,banned_by,category,clicked,content_categories,is_original_content,likes,link_flair_richtext,link_flair_text_color,link_flair_type,media_only,mod_reports,num_reports,post_categories,pwls,removal_reason,report_reasons,rte_mode,selftext_html,user_reports,visited,wls,author_flair_richtext,author_flair_type,previous_visits,author_created_utc,author_fullname,is_meta,link_flair_background_color,link_flair_template_id,subreddit_name_prefixed,author_patreon_flair,gildings,is_robot_indexable,all_awardings,total_awards_received,allow_live_comments,discussion_type,awarders,removed_by,removed_by_category,author_premium,treatment_tags,upvote_ratio,is_created_from_ads_ui,retrieved_utc,top_awarded_type,media_metadata,call_to_action,url_overridden_by_dest
0,False,NaN,t5_347wh,1.414327e+09,http://www.reddit.com/r/AnxietyDepression/comm...,None,NaN,0.0,1,0,1.0,2kd2vt,t3_2kd2vt,NaN,0,False,0.0,0,None,True,Win7er,NaN,/r/AnxietyDepression/comments/2kd2vt/what_is_s...,NaN,1.441104e+09,1.0,None,What do u do to feel a bit better- what strata...,0.0,self.AnxietyDepression,AnxietyDepression,"{'content': None, 'height': None, 'scrolling':...",None,2014-10-26 11:42:51,"{'content': None, 'height': None, 'media_domai...",What is something you do to help anxiety and d...,NaN,self,0.0,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,None,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN
1,False,NaN,t5_347wh,NaN,https://www.reddit.com/r/AnxietyDepression/com...,None,NaN,0.0,1,0,0.0,4jl19h,t3_4jl19h,NaN,0,False,0.0,0,None,True,[deleted],NaN,/r/AnxietyDepression/comments/4jl19h/might_be_...,NaN,1.466463e+09,1.0,None,[deleted],0.0,self.AnxietyDepression,AnxietyDepression,"{'content': None, 'height': None, 'scrolling':...",None,2016-05-16 13:01:02,"{'content': None, 'height': None, 'media_domai...",Might be depressed but totally has anxiety pro...,NaN,default,0.0,0.0,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,None,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN
2,False,NaN,t5_347wh,NaN,https://www.reddit.com/r/AnxietyDepression/com...,None,NaN,0.0,1,0,0.0,4vfhs7,t3_4vfhs7,NaN,0,False,0.0,1,None,True,Douglesby,NaN,/r/AnxietyDepression/comments/4vfhs7/i_wish_i_...,NaN,1.473004e+09,1.0,None,"This is a nice, empty area. I probably don't h...",0.0,self.AnxietyDepression,AnxietyDepression,"{'content': None, 'height': None, 'scrolling':...",None,2016-07-31 03:39:21,"{'content': None, 'height': None, 'media_domai...","I wish I was someone else, and I wish I knew h...",NaN,self,0.0,0.0,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN

In [5]:
## step4. feature selection : select only necessary columns

In [7]:
anxiety_depression_submissions =\
    anxiety_depression_submissions[['subreddit','author','over_18','link_flair_text','title','selftext','url','created_utc']]
anxiety_depression_submissions


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,AnxietyDepression,Win7er,False,NaN,What is something you do to help anxiety and d...,What do u do to feel a bit better- what strata...,http://www.reddit.com/r/AnxietyDepression/comm...,2014-10-26 11:42:51
1,AnxietyDepression,[deleted],False,NaN,Might be depressed but totally has anxiety pro...,[deleted],https://www.reddit.com/r/AnxietyDepression/com...,2016-05-16 13:01:02
2,AnxietyDepression,Douglesby,False,NaN,"I wish I was someone else, and I wish I knew h...","This is a nice, empty area. I probably don't h...",https://www.reddit.com/r/AnxietyDepression/com...,2016-07-31 03:39:21
3,AnxietyDepression,[deleted],False,NaN,I wish I could die.,[deleted],https://www.reddit.com/r/AnxietyDepression/com...,2016-08-22 11:30:02
4,AnxietyDepression,Mariaisgone,False,NaN,Is the Earth really flat?,,https://www.reddit.com/r/AnxietyDepression/com...,2016-09-08 05:31:11
...,...,...,...,...,...,...,...,...
21329,AnxietyDepression,thatpandabear13,False,NaN,its getting harder to fight the urges,"hey so I have anxiety since I can remember, st...",https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 16:33:40
21330,AnxietyDepression,pindu3,False,NaN,Relaxing videos to calm your mind.,Please check and joiin community [**r/Soothin...,https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 18:48:48
21331,AnxietyDepression,pindu3,False,NaN,Please check and joiin community r/SoothingNat...,,https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 18:49:49
21332,AnxietyDepression,Revolutionaryage69,False,NaN,Anxiety-panic attack for 6 days now,"\nOn a random Tuesday last week, i suddenly ha...",https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 20:56:49


In [8]:
## step5. basic pre-processing 

In [9]:
# remove blank body posts
anxiety_depression_submissions = anxiety_depression_submissions[anxiety_depression_submissions['selftext'].apply(lambda x : len(x)!=0)]
anxiety_depression_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,AnxietyDepression,Win7er,False,NaN,What is something you do to help anxiety and d...,What do u do to feel a bit better- what strata...,http://www.reddit.com/r/AnxietyDepression/comm...,2014-10-26 11:42:51
1,AnxietyDepression,[deleted],False,NaN,Might be depressed but totally has anxiety pro...,[deleted],https://www.reddit.com/r/AnxietyDepression/com...,2016-05-16 13:01:02
2,AnxietyDepression,Douglesby,False,NaN,"I wish I was someone else, and I wish I knew h...","This is a nice, empty area. I probably don't h...",https://www.reddit.com/r/AnxietyDepression/com...,2016-07-31 03:39:21
3,AnxietyDepression,[deleted],False,NaN,I wish I could die.,[deleted],https://www.reddit.com/r/AnxietyDepression/com...,2016-08-22 11:30:02
5,AnxietyDepression,StephanieLynn94,False,NaN,light therapy?,Has anyone tried or used light therapy to reli...,https://www.reddit.com/r/AnxietyDepression/com...,2016-09-08 16:31:06
...,...,...,...,...,...,...,...,...
21328,AnxietyDepression,Darkstarunholy,False,NaN,Realizing,I don’t want anyone that’s temporary to be in ...,https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 13:38:58
21329,AnxietyDepression,thatpandabear13,False,NaN,its getting harder to fight the urges,"hey so I have anxiety since I can remember, st...",https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 16:33:40
21330,AnxietyDepression,pindu3,False,NaN,Relaxing videos to calm your mind.,Please check and joiin community [**r/Soothin...,https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 18:48:48
21332,AnxietyDepression,Revolutionaryage69,False,NaN,Anxiety-panic attack for 6 days now,"\nOn a random Tuesday last week, i suddenly ha...",https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 20:56:49


In [10]:
## opt out '[deleted]','[removed]' only selftext 
opt_out_list = ['[deleted]','[removed]']
anxiety_depression_submissions[anxiety_depression_submissions['selftext'].isin(opt_out_list)]

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
1,AnxietyDepression,[deleted],False,NaN,Might be depressed but totally has anxiety pro...,[deleted],https://www.reddit.com/r/AnxietyDepression/com...,2016-05-16 13:01:02
3,AnxietyDepression,[deleted],False,NaN,I wish I could die.,[deleted],https://www.reddit.com/r/AnxietyDepression/com...,2016-08-22 11:30:02
6,AnxietyDepression,[deleted],False,NaN,Crazy Angry rant....wanting to heal,[deleted],https://www.reddit.com/r/AnxietyDepression/com...,2016-09-11 17:59:38
11,AnxietyDepression,[deleted],False,NaN,Struggling,[deleted],https://www.reddit.com/r/AnxietyDepression/com...,2016-09-20 22:52:41
12,AnxietyDepression,[deleted],False,NaN,Bad day....,[deleted],https://www.reddit.com/r/AnxietyDepression/com...,2016-09-21 22:50:47
...,...,...,...,...,...,...,...,...
21308,AnxietyDepression,[deleted],False,NaN,About ready to throw myself off this bridge!!!,[deleted],,2022-12-29 01:43:34
21310,AnxietyDepression,[deleted],False,NaN,I need help...,[deleted],,2022-12-29 07:34:53
21311,AnxietyDepression,[deleted],False,NaN,Does anyone have a complete block when it come...,[deleted],,2022-12-29 08:25:18
21314,AnxietyDepression,[deleted],False,NaN,hey..,[deleted],,2022-12-29 21:01:41


In [11]:
anxiety_depression_submissions = anxiety_depression_submissions[~anxiety_depression_submissions['selftext'].isin(opt_out_list)]
print(len(anxiety_depression_submissions))
anxiety_depression_submissions

14398


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,AnxietyDepression,Win7er,False,NaN,What is something you do to help anxiety and d...,What do u do to feel a bit better- what strata...,http://www.reddit.com/r/AnxietyDepression/comm...,2014-10-26 11:42:51
2,AnxietyDepression,Douglesby,False,NaN,"I wish I was someone else, and I wish I knew h...","This is a nice, empty area. I probably don't h...",https://www.reddit.com/r/AnxietyDepression/com...,2016-07-31 03:39:21
5,AnxietyDepression,StephanieLynn94,False,NaN,light therapy?,Has anyone tried or used light therapy to reli...,https://www.reddit.com/r/AnxietyDepression/com...,2016-09-08 16:31:06
8,AnxietyDepression,peanutinthesun,False,NaN,Bye bye Zoloft! Or- advice on dealing with bra...,I recently stopped taking Zoloft. I'm having p...,https://www.reddit.com/r/AnxietyDepression/com...,2016-09-18 03:30:41
9,AnxietyDepression,x-ajs,False,NaN,Feeling Lost,This is my first post on Reddit and it might b...,https://www.reddit.com/r/AnxietyDepression/com...,2016-09-19 12:13:29
...,...,...,...,...,...,...,...,...
21328,AnxietyDepression,Darkstarunholy,False,NaN,Realizing,I don’t want anyone that’s temporary to be in ...,https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 13:38:58
21329,AnxietyDepression,thatpandabear13,False,NaN,its getting harder to fight the urges,"hey so I have anxiety since I can remember, st...",https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 16:33:40
21330,AnxietyDepression,pindu3,False,NaN,Relaxing videos to calm your mind.,Please check and joiin community [**r/Soothin...,https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 18:48:48
21332,AnxietyDepression,Revolutionaryage69,False,NaN,Anxiety-panic attack for 6 days now,"\nOn a random Tuesday last week, i suddenly ha...",https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 20:56:49


In [12]:
# step6. final data quality check 

In [13]:
print(f'Author : {anxiety_depression_submissions.iloc[[0],:].author.tolist()[0]}')
print(f'Subreddit : {anxiety_depression_submissions.iloc[[0],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {anxiety_depression_submissions.iloc[[0],:].over_18.tolist()[0]}')
print(f'Posted : {anxiety_depression_submissions.iloc[[0],:].created_utc.tolist()[0]}')
print()
print(f'Title : {anxiety_depression_submissions.iloc[[0],:].title.tolist()[0]}')
print(f'Body : {anxiety_depression_submissions.iloc[[0],:].selftext.tolist()[0]}')
print()
print()
print()
print(f'Author : {anxiety_depression_submissions.iloc[[1],:].author.tolist()[0]}')
print(f'Subreddit : {anxiety_depression_submissions.iloc[[1],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {anxiety_depression_submissions.iloc[[1],:].over_18.tolist()[0]}')
print(f'Posted : {anxiety_depression_submissions.iloc[[1],:].created_utc.tolist()[0]}')
print()
print(f'Title : {anxiety_depression_submissions.iloc[[1],:].title.tolist()[0]}')
print(f'Body : {anxiety_depression_submissions.iloc[[1],:].selftext.tolist()[0]}')


Author : Win7er
Subreddit : AnxietyDepression
Teenager(True/False) : False
Posted : 2014-10-26 11:42:51

Title : What is something you do to help anxiety and depression?
Body : What do u do to feel a bit better- what stratagies have you tried that have helped?



Author : Douglesby
Subreddit : AnxietyDepression
Teenager(True/False) : False
Posted : 2016-07-31 03:39:21

Title : I wish I was someone else, and I wish I knew how to get through life without help, but I need it.
Body : This is a nice, empty area. I probably don't have to worry about this being seen. It will be rambling and scatter-brained, because that's what it is in my head.

The worst part is, I don't think anything I'll write here will be unique. 

I'm 31, going on 32 in September. I've lived my whole life hoping I could make myself into someone I can admire, and someone that my parents and brother could be proud of. I dream so fiercely, and I love so fully, and I want so badly, and I help anyone I can. 

I love writing.

In [25]:
# step7. save final pre-processed data into parquet
anxiety_depression_submissions.to_parquet('anxiety_depression_submissions_preprocessed.parquet', engine='pyarrow')
anxiety_depression_submissions_preprocessed = pd.read_parquet('anxiety_depression_submissions_preprocessed.parquet',engine='pyarrow')
anxiety_depression_submissions_preprocessed

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,AnxietyDepression,Win7er,False,NaN,What is something you do to help anxiety and d...,What do u do to feel a bit better- what strata...,http://www.reddit.com/r/AnxietyDepression/comm...,2014-10-26 11:42:51
2,AnxietyDepression,Douglesby,False,NaN,"I wish I was someone else, and I wish I knew h...","This is a nice, empty area. I probably don't h...",https://www.reddit.com/r/AnxietyDepression/com...,2016-07-31 03:39:21
5,AnxietyDepression,StephanieLynn94,False,NaN,light therapy?,Has anyone tried or used light therapy to reli...,https://www.reddit.com/r/AnxietyDepression/com...,2016-09-08 16:31:06
8,AnxietyDepression,peanutinthesun,False,NaN,Bye bye Zoloft! Or- advice on dealing with bra...,I recently stopped taking Zoloft. I'm having p...,https://www.reddit.com/r/AnxietyDepression/com...,2016-09-18 03:30:41
9,AnxietyDepression,x-ajs,False,NaN,Feeling Lost,This is my first post on Reddit and it might b...,https://www.reddit.com/r/AnxietyDepression/com...,2016-09-19 12:13:29
...,...,...,...,...,...,...,...,...
21328,AnxietyDepression,Darkstarunholy,False,NaN,Realizing,I don’t want anyone that’s temporary to be in ...,https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 13:38:58
21329,AnxietyDepression,thatpandabear13,False,NaN,its getting harder to fight the urges,"hey so I have anxiety since I can remember, st...",https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 16:33:40
21330,AnxietyDepression,pindu3,False,NaN,Relaxing videos to calm your mind.,Please check and joiin community [**r/Soothin...,https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 18:48:48
21332,AnxietyDepression,Revolutionaryage69,False,NaN,Anxiety-panic attack for 6 days now,"\nOn a random Tuesday last week, i suddenly ha...",https://www.reddit.com/r/AnxietyDepression/com...,2022-12-31 20:56:49
